In [1]:
%pip install -q langchain-ollama langchain langchain-community faiss-cpu langchain_huggingface rank_bm25 gradio nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [2]:
from re import search
# Custom MultiQuery with Output Parser
from typing import List
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_ollama import OllamaLLM


# Initialize the LLM
from langchain_ollama import OllamaLLM
multiquery_llm = OllamaLLM(model="llama3.2:latest", temperature=0.5)

# Initialize the embedding model
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Load the vector store
from langchain.vectorstores import FAISS
loaded_faiss_store = FAISS.load_local(
    "/workspaces/RAG_BOT/LocalEmbeddings/Hugging_split_enriched_faiss_index",
    embedding_model,
    allow_dangerous_deserialization=True
)
print("FAISS vector store loaded successfully.")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FAISS vector store loaded successfully.


In [3]:
# Ensemble Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

# Extract documents from the docstore
try:
    all_docs = [loaded_faiss_store.docstore._dict[doc_id] for doc_id in loaded_faiss_store.index_to_docstore_id.values()]
except AttributeError:
    # Fallback for different docstore structure
    all_docs = [loaded_faiss_store.docstore.get(doc_id) for doc_id in loaded_faiss_store.index_to_docstore_id.values()]


# Create BM25 retriever
bm25_retriever = BM25Retriever.from_documents(all_docs)
bm25_retriever.k = 2

In [4]:
basic_retriever = loaded_faiss_store.as_retriever(search_type="mmr", search_kwargs={"k": 2})

In [5]:
sst_retriever = loaded_faiss_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.3, "k": 2}
)

In [6]:
# Create ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, basic_retriever, sst_retriever],
    weights=[0.33, 0.33, 0.33]
)


In [7]:
from langchain_ollama import ChatOllama
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,
    check_every_n_seconds= 0.1,
    max_bucket_size = 10,
)

llm = ChatOllama(
    model = 'llama3.2:latest',
    temperature=0.1,
    rate_limiter= rate_limiter
)


In [8]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


SYSTEM_PROMPT = """
You are a highly knowledgeable CyberArk API documentation assistant. Your job is to answer developer questions accurately and clearly using only the provided API documentation context.

For general greetings or small talk (like "hello", "hi"), respond politely as a friendly assistant.

For Cyberark API documentation related questions, Your answers must follow these rules:

1. Use only the given context. If the answer is not in the context, say "I don't know based on the provided documentation."
2. If the user asks about an endpoint, provide its details from the context including:
   - Path and method
   - Required parameters (query, path, body)
   - Security requirements
   - Request body schema (in JSON if available)
   - Response body schema (in JSON if available)
   - Sample request and response if present
3. Be clear and structured:
   - Use bullet points for properties
   - Include code blocks for JSON
4. Never invent or guess missing details.
5. If the context includes multiple endpoints, select only the most relevant.
6. For CyberArk API questions, use only the given context. If the answer is not in the context, say "I don't know based on the provided documentation."

Answer as if you are the official CyberArk API documentation.
"""

system_message = SystemMessagePromptTemplate.from_template(SYSTEM_PROMPT)

human_message = HumanMessagePromptTemplate.from_template(
    """
You are answering questions about CyberArk's API. Use the documentation context

Documentation Context:
----------------------
{context}

New User Question:
----------------------
{question}
"""
)



In [9]:
# Function to create QA chain and get answer
def get_answer(retriever, query, description=""):
    from langchain.chains import RetrievalQA

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )

    # Test the prompt with a sample invocation
    sample_question = query
    sample_context = retriever.invoke(query)
    print(f"\n=== {description} ===")
    print(f"sample_question: {sample_question}")
    print("SAMPLE_CONTEXT")
    print(sample_context)
    CONVERSATIONAL_PROMPT = ChatPromptTemplate.from_messages([
     system_message,
     human_message
    ])

    print(CONVERSATIONAL_PROMPT)

    # Format the prompt with context and question before invoking the chain
    formatted_prompt = CONVERSATIONAL_PROMPT.format(
        context="\n\n".join([doc.page_content for doc in sample_context]),
        question=sample_question
    )

    result = qa_chain.invoke({"query": formatted_prompt})

    print("Answer:")
    print(result["result"])
    print(f"Number of source documents: {len(result['source_documents'])}")

    # return result

In [10]:

get_answer(ensemble_retriever, "what are radiusclientlist?", "Ensemble Retriever")


=== Ensemble Retriever ===
sample_question: what are radiusclientlist?
SAMPLE_CONTEXT
[Document(id='2642fdd1-88db-4c33-83aa-f4350365a1ad', metadata={}, page_content='cription: "The policy revision stamp.", type: "string" }, RadiusClientList: { description: "The list of radius clients.", type: "array", items: { type: "object" } } } }\n  Sample Response JSON:\n  ```json\n  {\n  "Result": {\n    "RiskAnalysisLevels": {},\n    "AuthProfiles": [\n      {}\n    ],\n    "PolicyModifiers": [\n      "string_value"\n    ],\n    "RevStamp": "string_value",\n    "RadiusClientList": [\n      {}\n    ]\n  },\n  "Error": {}\n}\n```'), Document(id='ee12bd3c-daec-49de-a15d-30e9edce370c', metadata={}, page_content='er authentication to access the authentication profile endpoint?\n* What if there\'s an error with the request - what will be returned in the response?\n* How can I get a new authentication profile?\n\n**Developer Notes**\n------------------\n\n### Required Parameters\n\n* `uuid`: The authen

KeyboardInterrupt: 

In [ ]:
def start_cli():
    """
    Simple command line interface for the CyberArk API documentation assistant.
    Uses the existing get_answer method with the ensemble retriever.
    """
    print("=" * 50)
    print("CyberArk API Documentation Assistant CLI")
    print("Type 'exit', 'quit', or 'q' to end the session")
    print("=" * 50)
    
    while True:
        # Get user input
        query = input("\nEnter your question: ")
        
        # Check if user wants to exit
        if query.lower() in ['exit', 'quit', 'q']:
            print("Thank you for using the CyberArk API Documentation Assistant!")
            break
            
        # If the query is not empty, process it
        if query.strip():
            # Call the existing get_answer function with ensemble_retriever
            get_answer(ensemble_retriever, query, "CLI Query")
        else:
            print("Please enter a valid question.")

# Run the CLI
start_cli()

CyberArk API Documentation Assistant CLI
Type 'exit', 'quit', or 'q' to end the session


No relevant docs were retrieved using the relevance score threshold 0.3



=== CLI Query ===
sample_question: what is radius client
SAMPLE_CONTEXT
[Document(id='f879b49d-0d31-40fd-a320-9e0331b0d039', metadata={}, page_content='lients associated with a policy?\n* Can I retrieve the policy modifiers for a given policy block?\n* What is the error response when the policy block does not exist?\n* How do I authenticate using bearer token for this endpoint?\n\n**Developer Notes**\n------------------\n\n### Required Parameters\n\n* `name` (query, Required): The policy block to get.\n\n### Request/Response Structure\n\nThe request body is empty. The response body contains a JSON object with two properties: `Result` and `Error`. The `Result` property contains the policy block details, including risk analysis mapping, authentication profiles, policy modifiers, revision stamp, and radius client list. The `Error` property contains an error message if the policy block does not exist.\n\n### Error Handling\n\n* If the policy block does not exist, an error response is retu